__качаем объявления о продаже недвижимости с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import logging
import sys 

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [2]:
from time import sleep
from tqdm.auto import tqdm
import pandas as pd
from datetime import datetime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

---

In [3]:
source = [
    {
            'place': 'sevastopol',
        'subfolder': 'flat',
              'url': 'https://sevastopol.cian.ru/kupit-kvartiru/',
    },{
            'place': 'sevastopol',
        'subfolder': 'land',
              'url': 'https://sevastopol.cian.ru/kupit-zemelniy-uchastok/',
    },{
            'place': 'sevastopol',
        'subfolder': 'house',
              'url': 'https://sevastopol.cian.ru/kupit-dom/',
    },{
            'place': 'sudak',
        'subfolder': 'flat',
              'url': 'https://krym.cian.ru/kupit-kvartiru-krym-sudak/',
    },{
            'place': 'sudak',
        'subfolder': 'land',
              'url': 'https://krym.cian.ru/kupit-zemelniy-uchastok-krym-sudak/',
    },{
            'place': 'sudak',
        'subfolder': 'house',
              'url': 'https://krym.cian.ru/kupit-dom-krym-sudak/',
    },    
#     {
#             'place': 'bahchisaray',
#         'subfolder': 'flat',
#               'url': 'https://krym.cian.ru/kupit-kvartiru-krym-bahchisaray/',
#     }, {
#             'place': 'bahchisaray',
#         'subfolder': 'land',
#               'url': 'https://krym.cian.ru/kupit-zemelniy-uchastok-krym-bahchisaray/',
#     },{
#             'place': 'bahchisaray',
#         'subfolder': 'house',
#               'url': 'https://krym.cian.ru/kupit-dom-krym-bahchisaray/',
#     },

]


---

In [4]:
# from selenium import webdriver
# from lib.cian import CianPageParser

# browser = webdriver.Firefox()
# browser.get(source[0]['url'])
# data = CianPageParser().transform(browser)
# browser.close()

In [5]:
# data
# browser.close()

---

In [6]:
dest_folder = 'data/2024/raw'

In [7]:
from selenium import webdriver
# from selenium.webdriver.common.by import By
from lib.cian import CianParser
# from lib.cian import CianDataCleaner

In [8]:
%%time 

#from selenium.webdriver.firefox.options import Options
#from selenium.webdriver.firefox.options import FirefoxProfile
# profile_path = '/home/mechanoid/.mozilla/firefox/4btdfxw1.default-release'
# options = Options()
# options.profile = FirefoxProfile(profile_path) 
# options.headless = True
# browser = webdriver.Firefox(options=options)

options = webdriver.FirefoxOptions()
options.headless = True
browser = webdriver.Firefox(options=options)
logging.info('open virtual browser')

for s in source:

    url = s['url']
    place = s['place']
    subfolder = s['subfolder'] 
    
    display(url)
    browser.get(url)
    
    data = CianParser().transform(browser)
    assert len(data)>0
    
    df = pd.DataFrame(data)
    df['ts'] = datetime.now()
    df['place'] = place
    
    display( len(df) )
    t_now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    fname = f'{dest_folder}/{subfolder}/cian/cian_{t_now}_raw_{place}.pkl'
    df.to_pickle(fname)
    display( fname )
       
browser.close()

[INFO    ] 2024-07-16 12:54:23 | open virtual browser


'https://krym.cian.ru/kupit-zemelniy-uchastok-krym-sudak/'

[INFO    ] 2024-07-16 12:54:29 | page 1: 28 items
[INFO    ] 2024-07-16 12:54:33 | page 2: 28 items
[INFO    ] 2024-07-16 12:54:37 | page 3: 28 items
[INFO    ] 2024-07-16 12:54:41 | page 4: 28 items
[INFO    ] 2024-07-16 12:54:45 | page 5: 28 items
[INFO    ] 2024-07-16 12:54:49 | page 6: 28 items
[INFO    ] 2024-07-16 12:54:50 | page 7: 0 items


168

'data/2024/raw/land/cian/cian_2024-07-16_12-54_raw_sudak.pkl'

'https://krym.cian.ru/kupit-dom-krym-sudak/'

[INFO    ] 2024-07-16 12:54:55 | page 1: 28 items
[INFO    ] 2024-07-16 12:54:59 | page 2: 28 items
[INFO    ] 2024-07-16 12:55:03 | page 3: 28 items
[INFO    ] 2024-07-16 12:55:09 | page 4: 28 items
[INFO    ] 2024-07-16 12:55:15 | page 5: 28 items
[INFO    ] 2024-07-16 12:55:21 | page 6: 28 items
[INFO    ] 2024-07-16 12:55:25 | page 7: 28 items
[INFO    ] 2024-07-16 12:55:29 | page 8: 28 items
[INFO    ] 2024-07-16 12:55:34 | page 9: 28 items
[INFO    ] 2024-07-16 12:55:38 | page 10: 28 items
[INFO    ] 2024-07-16 12:55:43 | page 11: 25 items


305

'data/2024/raw/house/cian/cian_2024-07-16_12-55_raw_sudak.pkl'

CPU times: user 3.56 s, sys: 123 ms, total: 3.68 s
Wall time: 1min 22s


In [9]:
!date

Вт 16 июл 2024 12:55:43 MSK


In [10]:
# browser.close()